In [2]:
import glob
import datetime
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from features import *

In [36]:
path = 'data/*.csv'
files = glob.glob(path)

In [18]:
pumps = pd.read_csv('pump_binance_telegram.csv')
pumps = pumps[pumps['exchange'] == 'binance']

In [19]:
f = files[0]
coin_date, time = os.path.basename(f[:f.rfind('.')]).split(' ')
coin, date = coin_date.split('_')
time_freq='25S'
rolling_freq=900
count = 0

In [20]:
#作者构建特征std_rush_order的过程
df = pd.read_csv(f)
df["time"] = pd.to_datetime(df['timestamp'].astype(np.int64), unit='ms')
df = df.reset_index().set_index('time')
df_buy = df[df['side'] == 'buy']
df_buy = df_buy.groupby(df_buy.index).count()
df_buy[df_buy == 1] = 0
df_buy[df_buy > 1] = 1
buy_volume = df_buy.groupby(pd.Grouper(freq=time_freq))['btc_volume'].sum()
buy_count = df_buy.groupby(pd.Grouper(freq=time_freq))['btc_volume'].count()
buy_volume.drop(buy_volume[buy_count == 0].index, inplace=True)
buy_volume.dropna(inplace=True)
rolling_diff = buy_volume.rolling(window=rolling_freq).std()
results = rolling_diff.pct_change()
display(results)
print(results.shape)

time
2019-01-21 18:00:50         NaN
2019-01-21 18:01:15         NaN
2019-01-21 18:12:30         NaN
2019-01-21 18:17:55         NaN
2019-01-21 18:18:45         NaN
                         ...   
2019-02-09 18:42:55    0.000000
2019-02-09 18:43:45   -0.002002
2019-02-09 18:44:35    0.000113
2019-02-09 18:47:30   -0.001120
2019-02-09 18:47:55   -0.000900
Name: btc_volume, Length: 13581, dtype: float64

(13581,)


In [130]:
#用于测试std_rush_order 特征构建
#根据列time 进行groupby ,然后对数据重采样(降采样)，计算降采样后的25Sbtc_volume 的和
#区别于作者构建的特征，作者构建的特征先根据时间索引(毫秒级别)进行groupby 然后count,
#凡是只出现一次的时间索引，都置为了0，他会把只出现一次的时间索引都丢掉
#作者这样做只考虑时间（精确到毫秒）相同的订单，只对时间相同的订单按照25秒进行合并
#也就是在25s内，凡是时间相同的

df = pd.read_csv(f)
df["time"] = pd.to_datetime(df['timestamp'].astype(np.int64), unit='ms')
df_buy = df[df['side'] == 'buy']
buy_volume = df_buy.groupby(pd.Grouper(key='time',freq=time_freq))['btc_volume'].sum()
buy_count = df_buy.groupby(pd.Grouper(key='time',freq=time_freq))['btc_volume'].count()
buy_volume.drop(buy_count[buy_count==0].index,inplace=True)
rolling_df = buy_volume.rolling(window=rolling_freq).std()
results.dropna(inplace=True)
results = rolling_df.pct_change()
results.reset_index().set_index('time')




,btc_volume
time,
2019-01-21 18:00:50,NaN
2019-01-21 18:01:15,NaN
2019-01-21 18:12:30,NaN
2019-01-21 18:17:55,NaN
2019-01-21 18:18:45,NaN
...,...
2019-02-09 18:42:55,-0.000042
2019-02-09 18:43:45,-0.000048
2019-02-09 18:44:35,0.000071


In [131]:
# 构建avg_rush_order_feature
df = pd.read_csv(f)
df["time"] = pd.to_datetime(df['timestamp'].astype(np.int64), unit='ms')
df_buy = df[df['side'] == 'buy']
buy_volume = df_buy.groupby(pd.Grouper(key='time',freq=time_freq))['btc_volume'].sum()
buy_count = df_buy.groupby(pd.Grouper(key='time',freq=time_freq))['btc_volume'].count()
buy_volume.drop(buy_volume[buy_count == 0].index, inplace=True)
buy_volume.dropna(inplace=True)
rolling_diff = buy_volume.rolling(window=rolling_freq).mean()
results = rolling_diff.pct_change()
results.reset_index().set_index('time')

,btc_volume
time,
2019-01-21 18:00:50,NaN
2019-01-21 18:01:15,NaN
2019-01-21 18:12:30,NaN
2019-01-21 18:17:55,NaN
2019-01-21 18:18:45,NaN
...,...
2019-02-09 18:42:55,0.000690
2019-02-09 18:43:45,-0.002341
2019-02-09 18:44:35,-0.000559


In [8]:
#读取特征文件
time_freq = '25S'
features_path = "features/features_{}.csv".format(time_freq)
df_features = pd.read_csv(features_path)
# df_features.set_index(pd.to_datetime(df_features['date']),inplace=True)
print("features shape",df_features.shape)

#读取25S 特征的标签文件
label_path = "labeled_features/label_{}.csv".format(time_freq)
df_label = pd.read_csv(label_path)
df_label = df_label.loc[:,['date','symbol','gt']]
print("label file gt == 1 shape:",df_label.shape)
# df_label_output = df_label[df_label['gt']==1]
# df_label.set_index(pd.to_datetime(df_label['date']),inplace=True)
# display(df_label.loc['2018-12-30 17:00:00'])

#合并特征文件和标签文件
output_path = 'labeled_features/my_features_{}.csv'.format(time_freq)
new_df = pd.merge(left=df_features,right=df_label,on=['symbol','date'],how='left')
new_df['gt'] = new_df['gt_y']
new_df.drop(columns=['gt_x','gt_y'],inplace=True)
new_df['gt'].fillna(inplace=True,value=0)
new_df.to_csv(output_path,index=False)
print("label == 1 shape:",new_df[new_df['gt']==1].shape)

features shape (538976, 16)
label file gt == 1 shape: (317, 3)
label == 1 shape: (337, 16)


,date,pump_index,std_rush_order,avg_rush_order,std_trades,std_volume,avg_volume,std_price,avg_price,avg_price_max,hour_sin,hour_cos,minute_sin,minute_cos,symbol,gt
224,2019-02-02 17:58:25,0,0.362,0.056,0.362,0.161,0.099,0.034,0.006,0.008,-0.998,-0.068,-0.106,0.994,GRS,1.0
1574,2020-08-02 16:00:05,1,3.078,0.146,3.078,1.920,0.295,0.026,0.012,0.020,-0.942,-0.335,0.000,1.000,NXS,1.0
2594,2020-03-21 16:00:20,2,0.002,0.003,0.002,0.004,0.020,0.008,0.010,0.010,-0.942,-0.335,0.000,1.000,TNB,1.0
3433,2020-12-15 18:01:10,3,0.106,0.034,0.106,0.005,0.020,0.004,0.001,0.003,-0.979,0.203,0.106,0.994,NEBL,1.0
5087,2020-12-20 18:01:50,4,0.125,0.039,0.125,0.000,0.002,0.006,0.000,-0.001,-0.979,0.203,0.106,0.994,VIB,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948194,2018-04-15 14:57:10,333,1.092,0.098,1.092,1.891,0.348,0.011,0.002,0.004,-0.631,-0.776,-0.211,0.977,OAX,1.0
951175,2018-10-13 19:00:00,334,0.228,0.027,0.228,0.376,0.088,-0.000,0.002,0.004,-0.888,0.460,0.000,1.000,EDO,1.0
953908,2019-06-16 21:00:15,335,3.676,0.210,3.676,17.430,1.431,0.049,0.017,0.034,-0.520,0.854,0.000,1.000,NEBL,1.0
956499,2019-05-13 15:00:00,336,0.377,0.037,0.377,0.177,0.053,-0.000,0.011,0.022,-0.817,-0.577,0.000,1.000,CDT,1.0


,date,pump_index,std_rush_order,avg_rush_order,std_trades,std_volume,avg_volume,std_price,avg_price,avg_price_max,hour_sin,hour_cos,minute_sin,minute_cos,symbol,gt
0,2019-02-01 18:20:50,0,0.0,-0.000,0.0,-0.0,0.000,0.000,-0.000,-0.000,-0.979,0.203,0.848,-0.530,GRS,0.0
1,2019-02-01 18:32:05,0,0.0,0.000,0.0,-0.0,0.000,0.000,0.000,0.000,-0.979,0.203,-0.263,-0.965,GRS,0.0
2,2019-02-01 18:50:50,0,0.0,0.000,0.0,0.0,-0.000,0.000,-0.000,-0.000,-0.979,0.203,-0.818,0.575,GRS,0.0
3,2019-02-01 18:53:45,0,-0.0,0.000,-0.0,0.0,-0.001,0.000,0.001,0.001,-0.979,0.203,-0.596,0.803,GRS,0.0
4,2019-02-01 18:54:10,0,0.0,0.000,0.0,0.0,-0.000,0.000,0.001,0.001,-0.979,0.203,-0.508,0.862,GRS,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538971,2018-12-08 16:44:10,337,0.0,0.000,0.0,0.0,-0.001,-0.003,0.000,0.000,-0.942,-0.335,-1.000,-0.027,OAX,0.0
538972,2018-12-08 16:45:00,337,-0.0,0.001,-0.0,-0.0,0.000,-0.003,0.001,0.001,-0.942,-0.335,-0.997,0.080,OAX,0.0
538973,2018-12-08 16:46:40,337,0.0,0.000,0.0,0.0,-0.001,-0.003,0.000,0.000,-0.942,-0.335,-0.983,0.185,OAX,0.0
538974,2018-12-08 16:55:50,337,-0.0,0.000,-0.0,0.0,0.001,-0.003,0.000,0.000,-0.942,-0.335,-0.413,0.911,OAX,0.0


## 测试pd.merge

In [ ]:
df = pd.DataFrame({'symbol':[1,2,3,4,5,6],'date':[1,2,3,4,5,6],'value':[0,0,0,0,0,0]})
df2 = pd.DataFrame({'symbol':[5,6,7,8,9,10],'date':[5,6,7,8,9,10],'value':[1,1,1,1,1,1]})
pd.merge(left=df,right=df2,on=['symbol','date'],how='left')

# 测试各种特征是否包含nan

In [24]:
files = glob.glob(path)
all_results_df = pd.DataFrame()
count = 0
pumps = pd.read_csv('pump_telegram.csv')
pumps = pumps[pumps['exchange'] == 'binance']



df = pd.read_csv(files[0])
df["time"] = pd.to_datetime(df['timestamp'].astype(np.int64), unit='ms')
df = df.reset_index().set_index('time')

df_buy = df[df['side'] == 'buy']

df_buy_grouped = df_buy.groupby(pd.Grouper(freq=time_freq))

#获取groupby 后的索引，将时间序列按照25s的周期进行重采样
#该函数只是返回索引
date = chunks_time(df_buy_grouped)

In [36]:
df = pd.DataFrame({'A':[1,2,None,np.nan],'B':[1,2,3,np.nan]})

In [38]:
df.count()

A    2
B    3
dtype: int64

# 如何根据pump 和dump datetime 生成标签文件

In [5]:
time_freq = '25S'
df1 = pd.read_csv('labeled_features/features_' + time_freq + '.csv.gz', parse_dates=['date'])
time_freq = '15S'
df2 = pd.read_csv('labeled_features/features_' + time_freq + '.csv.gz', parse_dates=['date'])
time_freq = '5S'
df3 = pd.read_csv('labeled_features/features_' + time_freq + '.csv.gz', parse_dates=['date'])

In [6]:
df1_label = df1.loc[:,['date','symbol','gt']]
df2_label = df2.loc[:,['date','symbol','gt']]
df3_label = df3.loc[:,['date','symbol','gt']]

In [7]:
df1_label[ (df1_label['symbol']=='BRD') & (df1_label['gt']==1)]


,date,symbol,gt
553,2018-12-30 17:00:00,BRD,1
10276,2019-10-03 14:59:35,BRD,1
33908,2020-11-09 16:00:00,BRD,1
50552,2019-05-25 16:00:25,BRD,1
57141,2019-06-17 14:59:10,BRD,1
104135,2018-11-13 17:00:00,BRD,1
172110,2019-04-14 19:00:50,BRD,1
263480,2019-03-17 19:00:25,BRD,1
322342,2020-03-24 15:58:45,BRD,1
330601,2018-10-04 18:58:45,BRD,1


In [ ]:
df2_label[ (df2_label['symbol']=='BRD') & (df2_label['gt']==1)]


In [79]:
df3_label[ (df3_label['symbol']=='BRD') & (df3_label['gt']==1)]


,date,symbol,gt
794,2018-12-30 16:59:55,BRD,1
16203,2019-10-03 14:59:45,BRD,1
59950,2020-11-09 16:00:00,BRD,1
85509,2019-05-25 16:01:05,BRD,1
97817,2019-06-17 14:59:25,BRD,1
177334,2018-11-13 17:00:00,BRD,1
291494,2019-04-14 19:00:55,BRD,1
435979,2019-03-17 19:00:40,BRD,1
542541,2020-03-24 15:58:45,BRD,1
556640,2018-10-04 18:59:05,BRD,1


In [30]:
data = pd.date_range(start='2022-01-01 00:00:00',end = '2022-01-01 01:00:00',freq='s')
display(data.shape)

(3601,)

In [31]:
df = pd.DataFrame({'date':data,'value':range(3601)})

In [34]:
df.groupby('date').sum()

,value
date,
2022-01-01 00:00:00,0
2022-01-01 00:00:01,1
2022-01-01 00:00:02,2
2022-01-01 00:00:03,3
2022-01-01 00:00:04,4
...,...
2022-01-01 00:59:56,3596
2022-01-01 00:59:57,3597
2022-01-01 00:59:58,3598


## 构建新的特征变量

### 最大值和最小值的差
该特征添加完后，模型性能没有提高

In [54]:
def avg_diff_min_max(df_buy_grouped,rolling_freq):
    """
        计算最大值和最小值的移动差的平均值
        添加该特征后，模型性能没有提高
    @param df_buy:
    @return:
    """
    results = df_buy_grouped['price'].max() - df_buy_grouped['price'].min()
    results.dropna(inplace=True)
    rolling_diff = results.rolling(window=rolling_freq).mean()
    return rolling_diff.pct_change()

In [41]:
path = 'data/*.csv'
files = glob.glob(path)
file = files[0]
time_freq='15S'
df = pd.read_csv(file)
df["time"] = pd.to_datetime(df['timestamp'].astype(np.int64), unit='ms')
df = df.reset_index().set_index('time')
df_buy = df[df['side'] == 'buy']
df_buy_grouped = df_buy.groupby(pd.Grouper(freq=time_freq))

In [55]:
avg_diff_min_max(df_buy_grouped,rolling_freq=240)

time
2019-01-21 18:01:00         NaN
2019-01-21 18:01:30         NaN
2019-01-21 18:12:30         NaN
2019-01-21 18:12:45         NaN
2019-01-21 18:18:00         NaN
                         ...   
2019-02-09 18:42:45    0.025608
2019-02-09 18:43:45    0.000000
2019-02-09 18:44:30    0.000000
2019-02-09 18:47:30    0.022472
2019-02-09 18:48:00    0.000000
Name: price, Length: 17064, dtype: float64

In [52]:
features.dropna(inplace=True)
features

time
2019-01-21 18:01:00    0.000000e+00
2019-01-21 18:01:30    0.000000e+00
2019-01-21 18:12:30    6.000000e-08
2019-01-21 18:12:45    0.000000e+00
2019-01-21 18:18:00    1.000000e-08
                           ...     
2019-02-09 18:42:45    2.000000e-07
2019-02-09 18:43:45    0.000000e+00
2019-02-09 18:44:30    0.000000e+00
2019-02-09 18:47:30    1.800000e-07
2019-02-09 18:48:00    0.000000e+00
Name: price, Length: 17064, dtype: float64

## 什么是rush order
将同一个时间戳的数据进行合并(同一毫秒)

In [60]:
path = 'data/*.csv'
files = glob.glob(path)
file = files[1]
time_freq='15S'
df = pd.read_csv(file)

In [61]:
df.head()

,symbol,timestamp,datetime,side,price,amount,btc_volume
0,NXS/BTC,1595347204587,2020-07-21T16:00:04.587Z,buy,0.000025,573.0,0.014589
1,NXS/BTC,1595347206246,2020-07-21T16:00:06.246Z,sell,0.000025,5.0,0.000127
2,NXS/BTC,1595347207065,2020-07-21T16:00:07.065Z,buy,0.000025,9.0,0.000229
3,NXS/BTC,1595347208393,2020-07-21T16:00:08.393Z,buy,0.000025,9.0,0.000229
4,NXS/BTC,1595347208728,2020-07-21T16:00:08.728Z,buy,0.000025,3099.0,0.078932


In [62]:
df.groupby('side').count()

,symbol,timestamp,datetime,price,amount,btc_volume
side,,,,,,
buy,27767,27767,27767,27767,27767,27767
sell,24796,24796,24796,24796,24796,24796


## 能否添加sell 类型的特征

In [150]:
path = 'data/*.csv'
time_freq='25S'
rolling_freq=900
files = glob.glob(path)
file = files[1]
time_freq='15S'
df = pd.read_csv(file)
df["time"] = pd.to_datetime(df['timestamp'].astype(np.int64), unit='ms')
df = df.reset_index().set_index('time')

df_buy = df[df['side']=='buy']
df_sell = df[df['side']=='sell']
df_buy = df_buy.iloc[:df_sell.shape[0],:]
df_buy_grouped = df_buy.groupby(pd.Grouper(freq=time_freq))
df_sell_grouped = df_sell.groupby(pd.Grouper(freq=time_freq))
df_grouped = df.groupby(pd.Grouper(freq=time_freq))
date = chunks_time(df_grouped)

In [132]:
df_buy.shape

(24796, 8)

In [133]:
df_sell.shape

(24796, 8)

In [141]:
features1 = std_rush_order_feature(df,time_freq,rolling_freq=rolling_freq).values
features1.shape

(13944,)

In [143]:
features2 = avg_rush_order_feature(df,time_freq,rolling_freq).values
features2.shape

(13944,)

In [147]:
features3 = avg_volume_feature(df_grouped,rolling_freq).values
features3.shape

(13944,)

In [148]:
features4 = std_price_feature(df_grouped,rolling_freq).values
features4.shape

(13944,)

In [149]:
features1 = std_price_feature(df_grouped,rolling_freq=rolling_freq).values
features1.shape

(13944,)

In [202]:
df_buy_volume = df[df['side']=='buy'].groupby(pd.Grouper(freq=time_freq))[['btc_volume','price']].sum()
df_buy_volume['index'] = df_buy_volume.index
df_buy_volume.dropna(inplace=True)
df_buy_volume.shape

(109506, 3)

In [203]:
df_sell_volume = df[df['side']=='sell'].groupby(pd.Grouper(freq=time_freq))[['btc_volume','price']].sum()
df_sell_volume['index'] = df_sell_volume.index
df_sell_volume.dropna(inplace=True)
df_sell_volume.shape

(109453, 3)

In [205]:
df_buy_volume

,btc_volume,price,index
time,,,
2020-07-21 16:00:00,11.563362,0.007525,2020-07-21 16:00:00
2020-07-21 16:00:15,6.579903,0.004804,2020-07-21 16:00:15
2020-07-21 16:00:30,11.199234,0.005086,2020-07-21 16:00:30
2020-07-21 16:00:45,3.015710,0.003017,2020-07-21 16:00:45
2020-07-21 16:01:00,2.156007,0.002685,2020-07-21 16:01:00
...,...,...,...
2020-08-09 16:15:15,0.000000,0.000000,2020-08-09 16:15:15
2020-08-09 16:15:30,0.000000,0.000000,2020-08-09 16:15:30
2020-08-09 16:15:45,0.000000,0.000000,2020-08-09 16:15:45


In [209]:
pd.merge(left=df_buy_volume,right = df_sell_volume,how='inner',on='index')

,btc_volume_x,price_x,index,btc_volume_y,price_y
0,11.563362,0.007525,2020-07-21 16:00:00,0.177888,0.000656
1,6.579903,0.004804,2020-07-21 16:00:15,4.236038,0.003904
2,11.199234,0.005086,2020-07-21 16:00:30,5.179718,0.004022
3,3.015710,0.003017,2020-07-21 16:00:45,3.630518,0.002176
4,2.156007,0.002685,2020-07-21 16:01:00,2.271517,0.002944
...,...,...,...,...,...
109448,0.000000,0.000000,2020-08-09 16:02:00,0.000000,0.000000
109449,0.000000,0.000000,2020-08-09 16:02:15,0.000000,0.000000
109450,0.000000,0.000000,2020-08-09 16:02:30,0.000000,0.000000
109451,0.000344,0.000021,2020-08-09 16:02:45,0.000322,0.000021


In [212]:
pd.merge(left=df_buy_volume,right = df_sell_volume,left_index=True,right_index=True)

,btc_volume_x,price_x,index_x,btc_volume_y,price_y,index_y
time,,,,,,
2020-07-21 16:00:00,11.563362,0.007525,2020-07-21 16:00:00,0.177888,0.000656,2020-07-21 16:00:00
2020-07-21 16:00:15,6.579903,0.004804,2020-07-21 16:00:15,4.236038,0.003904,2020-07-21 16:00:15
2020-07-21 16:00:30,11.199234,0.005086,2020-07-21 16:00:30,5.179718,0.004022,2020-07-21 16:00:30
2020-07-21 16:00:45,3.015710,0.003017,2020-07-21 16:00:45,3.630518,0.002176,2020-07-21 16:00:45
2020-07-21 16:01:00,2.156007,0.002685,2020-07-21 16:01:00,2.271517,0.002944,2020-07-21 16:01:00
...,...,...,...,...,...,...
2020-08-09 16:02:00,0.000000,0.000000,2020-08-09 16:02:00,0.000000,0.000000,2020-08-09 16:02:00
2020-08-09 16:02:15,0.000000,0.000000,2020-08-09 16:02:15,0.000000,0.000000,2020-08-09 16:02:15
2020-08-09 16:02:30,0.000000,0.000000,2020-08-09 16:02:30,0.000000,0.000000,2020-08-09 16:02:30


,index,symbol,timestamp,datetime,side,price,amount,btc_volume
time,,,,,,,,
2020-07-21 16:00:04.587,0,NXS/BTC,1595347204587,2020-07-21T16:00:04.587Z,buy,0.000025,573.0,0.014589
2020-07-21 16:00:07.065,2,NXS/BTC,1595347207065,2020-07-21T16:00:07.065Z,buy,0.000025,9.0,0.000229
2020-07-21 16:00:08.393,3,NXS/BTC,1595347208393,2020-07-21T16:00:08.393Z,buy,0.000025,9.0,0.000229
2020-07-21 16:00:08.728,4,NXS/BTC,1595347208728,2020-07-21T16:00:08.728Z,buy,0.000025,3099.0,0.078932
2020-07-21 16:00:08.728,5,NXS/BTC,1595347208728,2020-07-21T16:00:08.728Z,buy,0.000025,78.0,0.001987
...,...,...,...,...,...,...,...,...
2020-08-09 15:52:03.958,52556,NXS/BTC,1596988323958,2020-08-09T15:52:03.958Z,buy,0.000021,195.0,0.004191
2020-08-09 15:53:27.903,52557,NXS/BTC,1596988407903,2020-08-09T15:53:27.903Z,buy,0.000021,5.0,0.000107
2020-08-09 16:02:57.947,52558,NXS/BTC,1596988977947,2020-08-09T16:02:57.947Z,buy,0.000021,16.0,0.000344
